# 1. Initialisation

Importations

In [10]:
import numpy as np
import matplotlib
matplotlib.use("TkAgg")  # Ou "Qt5Agg"
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random

Paramètres du modèle

In [11]:
SIM_DURATION = 1000

WINDOW_SIZE = 7
DISTANCE = 5
BEAM_ANGLE = 0.2

DT = 1
NOISE = 0.1

RESOLUTION = 0.005
RADIUS_DECREASE_COEF = 0.5
RADIUS_INCREASE_COEF = 2

STEP_COEF = np.sqrt(2)

beam_radius = 0
power = 0

In [12]:
target_position = np.array([np.random.uniform(WINDOW_SIZE/4, 3*WINDOW_SIZE/4), np.random.uniform(WINDOW_SIZE/4, 3*WINDOW_SIZE/4)])
# target_speed = np.array([np.random.uniform(-0.1, 0.1), np.random.uniform(-0.1, 0.1)])
target_speed = np.array([0, 0])
# target_position = np.array([WINDOW_SIZE, WINDOW_SIZE])    # Pour écarter l'AUV du centre
sensor_position = np.array([WINDOW_SIZE/2, WINDOW_SIZE/2])
tracking_position = np.copy(sensor_position)

target_history = [target_position.copy()]
tracking_history = [tracking_position.copy()]

is_target_found = False

search_path = [(0, 0)] * SIM_DURATION

#Paramètres du déplacement du capteur
initial_axis = 0
axis = 0
direction = 1
side_length = 1
current_length = 0

Initialisation du plot

In [13]:
fig, ax = plt.subplots()
ax.set_xlim(0, WINDOW_SIZE)
ax.set_ylim(0, WINDOW_SIZE)
target_line, = ax.plot([], [], 'r--', alpha=0.5, label="Trajectoire cible")
tracking_line, = ax.plot([], [], 'b--', alpha=0.5, label="Trajectoire suivi")
sensor_dot, = ax.plot([], [], 'go', markersize=8, label="Capteur")
target_dot, = ax.plot([], [], 'ro', markersize=8, label="Cible")
tracking_dot, = ax.plot([], [], 'bo', markersize=8, label="Tracking")
ax.legend()
ax.set_title("Simulation Pointing, Acquisition, and Tracking (PAT)")

Text(0.5, 1.0, 'Simulation Pointing, Acquisition, and Tracking (PAT)')

# 2. PAT

A. Outils

In [14]:
def calculate_radius(distance, angle):
    """Calcule le rayon d'un faisceau à une certaine distance de la source"""
    radius = distance * np.tan(angle)
    return radius


def calculate_distance(point1, point2):
    """Calcule la distance entre la cible et le centre du faisceau"""
    distance = np.linalg.norm(point1 - point2)
    return distance


def detect_target():
    """Teste si la cible est dans le rayon du faisceau"""
    distance = calculate_distance(tracking_position, target_position)
    return distance <= beam_radius

B. Calcul du parcours du faisceau

In [15]:
def initialise_spiral_parameters(ax, dir):
    """Initialise les paramètres d'un déplacement en spirale commençant sur l'axe ax dans la direction dir"""
    global initial_axis, axis, direction, side_length, current_length
    initial_axis = ax
    axis = ax
    direction = dir
    side_length = 1
    current_length = 0


def move_sensor_spiral():
    """Déplace le capteur le long de la spirale"""
    global tracking_position, axis, direction, side_length, current_length
    point = tracking_position.copy()
    step = STEP_COEF * beam_radius
    if current_length < side_length: # si le segment de spirale n'est pas fini, on continue dessus
        current_length += 1
    else: # si l'on arrive sur un coin, on tourne
        if axis != initial_axis:
            direction *= -1
            side_length += 1
        axis = 1 - axis
        current_length = 1

    point[axis] += direction * step
    tracking_position = np.clip(point.copy(), 0, WINDOW_SIZE)


def check_neighbourhood():

    print(beam_radius)
    center = tracking_position
    closest_point = center.copy()
    min_dist = calculate_distance(closest_point, target_position)

    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            center = sensor_position.copy()
            directional_vector = np.array([dx, dy])
            temp_position = center + beam_radius * directional_vector
            distance = calculate_distance(temp_position, target_position)
            if distance < min_dist:
                closest_point = temp_position.copy()
                min_dist = distance
                # print(min_dist)
    # print(closest_point)
    # print(tracking_position)
    # print(target_position)
    # print(beam_radius)
    return closest_point


C. Calcul des déplacements des éléments

In [16]:
def move_target():
    """Met à jour les position et vitesse de la cible"""
    global target_position
    global target_speed

    x, y = target_position
    vx, vy = target_speed

    new_x = x + vx*DT + random.gauss(0, NOISE*vx*DT)
    new_y = y + vy*DT + random.gauss(0, NOISE*vy*DT)
    new_vx = (new_x - x) / DT
    new_vy = (new_y - y) / DT
    
    target_position = np.array([new_x, new_y])
    target_speed = np.array([new_vx, new_vy])
    target_position = np.clip(target_position, 0, WINDOW_SIZE)
    target_speed = np.clip(target_speed, 0, WINDOW_SIZE)

    # Limiter la taille de l'historique pour éviter une surcharge mémoire
    if len(target_history) > 50:
        target_history.pop(0)
    target_history.append(target_position.copy())


def search_target(frame):
    """Met à jour la position d'un faisceau cherchant la cible"""
    global tracking_position, is_target_found, beam_radius
    # tracking_position = np.clip(search_path[frame], 0, WINDOW_SIZE)
    move_sensor_spiral()
    # print("search")

    if len(tracking_history) > 50:
        tracking_history.pop(0)
    tracking_history.append(tracking_position.copy())

    if detect_target():
        if beam_radius <= calculate_radius(DISTANCE, RESOLUTION):
            is_target_found = True
            print("found")
        else:
            beam_radius *= RADIUS_DECREASE_COEF
            # print("decrease")
            initialise_spiral_parameters(axis, direction)


def track_target(frame):
    """Met à jour la position d'un faisceau traquant une cible qu'il a déjà trouvé"""
    global tracking_position, beam_radius
    
    if len(tracking_history) > 50:
        tracking_history.pop(0)
    tracking_history.append(tracking_position.copy())
    
    if not detect_target():
        print("increase")
        beam_radius *= RADIUS_INCREASE_COEF
    else:
        # print("tracking")
        tracking_position = check_neighbourhood()
        beam_radius *= RADIUS_DECREASE_COEF

    # if not detect_target():
    #     is_target_found = False
    #     # make_square_spiral_path(frame, beam_radius)
    #     # beam_radius *= RADIUS_INCREASE_COEF
    #     beam_radius = calculate_radius(DISTANCE, BEAM_ANGLE)
    #     initialise_spiral_parameters(axis, direction)


D. Mise à jour de l'animation

In [17]:
def update(frame):
    """Détermine l'état de la frame suivante"""
    global tracking_position, target_position

    if frame % 5 == 0:
        move_target()
    if is_target_found:
        track_target(frame)        
    else :
        search_target(frame)

    # Mise à jour des données des objets graphiques
    target_x, target_y = zip(*target_history)
    tracking_x, tracking_y = zip(*tracking_history)

    target_line.set_data(target_x, target_y)
    tracking_line.set_data(tracking_x, tracking_y)
    target_dot.set_data([target_position[0]], [target_position[1]])
    sensor_dot.set_data([sensor_position[0]], [sensor_position[1]])
    tracking_dot.set_data([tracking_position[0]], [tracking_position[1]])

    return target_line, target_dot, sensor_dot, tracking_line, tracking_dot,

E. Exécution

In [18]:

beam_radius = calculate_radius(DISTANCE, BEAM_ANGLE)
print(beam_radius)
initialise_spiral_parameters(axis, direction)
# make_square_spiral_path(0, beam_radius)

ani = animation.FuncAnimation(fig, update, frames=1000, interval=100, blit=True)
plt.show()

1.0135501775433626
